# Gem-Flux MCP Server: Basic Workflow

This notebook demonstrates the complete workflow for building, gapfilling, and analyzing a metabolic model using the Gem-Flux MCP Server.

## Workflow Overview

1. **Build Media** - Create growth medium from compounds
2. **Build Model** - Build draft metabolic model from protein sequences
3. **Gapfill Model** - Add reactions to enable growth
4. **Run FBA** - Analyze flux distribution and growth rate

## Prerequisites

- Gem-Flux MCP Server installed and running
- Python 3.11 environment
- ModelSEEDpy (Fxe/dev fork)
- COBRApy

## Setup

First, let's import the necessary modules and initialize the MCP tools.

In [1]:
# Import notebook setup helper
from notebook_setup import quick_setup

# Import Gem-Flux MCP tools
from gem_flux_mcp.tools.build_model import build_model
from gem_flux_mcp.tools.gapfill_model import gapfill_model
from gem_flux_mcp.tools.run_fba import run_fba

print("✓ Imports successful")

modelseedpy 0.4.3
✓ Imports successful


## Initialize Database and Templates

Load the ModelSEED database and templates required for model building.

In [2]:
# Setup environment using notebook helper
# This automatically:
# - Changes to repo root directory
# - Clears session storage
# - Loads database, templates, ATP media, and predefined media
# - All with correct paths

db_index, templates, atp_media, predefined_media = quick_setup()

print("\n✓ Environment ready!")
print(f"  Database: {db_index.get_compound_count()} compounds, {db_index.get_reaction_count()} reactions")
print(f"  Templates: {list(templates.keys())}")
print(f"  Predefined media: {list(predefined_media.keys())}")

✓ Session storage cleared
✓ Changed to repository root: /Users/jplfaria/repos/gem-flux-mcp

Loading database from data/database...
✓ Loaded 33992 compounds and 43774 reactions

Loading templates from data/templates...


Optional template 'GramPositive' not found at data/templates/GramPosModelTemplateV6.json, skipping


✓ Loaded 2 templates: ['GramNegative', 'Core']

Loading ATP gapfilling media...
✓ Loaded 54 ATP test media

Loading predefined media library...
✓ Loaded 4 predefined media: ['pyruvate_minimal_anaerobic', 'glucose_minimal_aerobic', 'pyruvate_minimal_aerobic', 'glucose_minimal_anaerobic']
✓ Stored 4 predefined media in session

Environment setup complete! Ready to use.

✓ Environment ready!
  Database: 33992 compounds, 43774 reactions
  Templates: ['GramNegative', 'Core']
  Predefined media: ['pyruvate_minimal_anaerobic', 'glucose_minimal_aerobic', 'pyruvate_minimal_aerobic', 'glucose_minimal_anaerobic']


## Step 1: Build Media

Create a minimal glucose medium for aerobic growth. This medium contains:
- D-Glucose (carbon source)
- O2 (electron acceptor)
- Essential minerals and ions

We'll use the predefined `glucose_minimal_aerobic` media for convenience.

In [3]:
# Option 1: Use predefined media (recommended)
media_id = "glucose_minimal_aerobic"
print(f"Using predefined media: {media_id}")
print("✓ Media ready")

# Option 2: Create custom media
# Uncomment the following to create custom media instead:
"""
custom_media_request = BuildMediaRequest(
    compounds=[
        "cpd00027",  # D-Glucose
        "cpd00007",  # O2
        "cpd00001",  # H2O
        "cpd00009",  # Phosphate
        "cpd00011",  # CO2
        "cpd00067",  # H+
        "cpd00013",  # NH3
        "cpd00048",  # SO4
        "cpd00205",  # K+
        "cpd00254",  # Mg
        "cpd00971",  # Na+
        "cpd00063",  # Ca2+
        "cpd00099",  # Cl-
        "cpd10515",  # Fe2+
        "cpd00030",  # Mn2+
        "cpd00034",  # Zn2+
        "cpd00058",  # Cu2+
        "cpd00149"   # Co2+
    ],
    default_uptake=100.0,
    custom_bounds={
        "cpd00027": (-5.0, 100.0),   # Limit glucose uptake
        "cpd00007": (-10.0, 100.0)   # Aerobic conditions
    }
)

media_response = build_media(custom_media_request, db_index)
media_id = media_response.media_id

print(f"\n✓ Created custom media: {media_id}")
print(f"  Compounds: {media_response.num_compounds}")
print(f"  Type: {media_response.media_type}")
"""

Using predefined media: glucose_minimal_aerobic
✓ Media ready


'\ncustom_media_request = BuildMediaRequest(\n    compounds=[\n        "cpd00027",  # D-Glucose\n        "cpd00007",  # O2\n        "cpd00001",  # H2O\n        "cpd00009",  # Phosphate\n        "cpd00011",  # CO2\n        "cpd00067",  # H+\n        "cpd00013",  # NH3\n        "cpd00048",  # SO4\n        "cpd00205",  # K+\n        "cpd00254",  # Mg\n        "cpd00971",  # Na+\n        "cpd00063",  # Ca2+\n        "cpd00099",  # Cl-\n        "cpd10515",  # Fe2+\n        "cpd00030",  # Mn2+\n        "cpd00034",  # Zn2+\n        "cpd00058",  # Cu2+\n        "cpd00149"   # Co2+\n    ],\n    default_uptake=100.0,\n    custom_bounds={\n        "cpd00027": (-5.0, 100.0),   # Limit glucose uptake\n        "cpd00007": (-10.0, 100.0)   # Aerobic conditions\n    }\n)\n\nmedia_response = build_media(custom_media_request, db_index)\nmedia_id = media_response.media_id\n\nprint(f"\n✓ Created custom media: {media_id}")\nprint(f"  Compounds: {media_response.num_compounds}")\nprint(f"  Type: {media_respo

## Step 2: Build Model

Build a draft metabolic model from E. coli protein sequences using the GramNegative template.

We'll create a small example with 5 key glycolysis proteins.

In [4]:
# Build model from E. coli FASTA file with RAST annotation
# This uses the real E. coli K-12 proteome (~4300 proteins)

print("Building model from E. coli proteome...")
print("This will take 2-5 minutes for RAST annotation and model construction.\n")

build_response = await build_model(
    fasta_file_path="examples/ecoli_proteins.fasta",
    template="GramNegative",
    model_name="E_coli_K12",
    annotate_with_rast=True  # Use RAST for functional annotation
)

model_id = build_response["model_id"]

# Show RAST annotation status
if build_response.get("annotated_with_rast"):
    print("✓ Genome annotated with RAST successfully\n")

print(f"✓ Model built: {model_id}")
print(f"  Reactions: {build_response['num_reactions']}")
print(f"  Metabolites: {build_response['num_metabolites']}")
print(f"  Genes: {build_response['num_genes']}")
print(f"  Exchange reactions: {build_response['num_exchange_reactions']}")
print(f"  Template: {build_response['template_used']}")
print(f"  Compartments: {build_response['compartments']}")
print(f"  Has biomass reaction: {build_response['has_biomass_reaction']}")
print(f"\nNote: This is a DRAFT model (suffix: .draft)")
print("      It likely cannot predict growth without gapfilling.")

Building model from E. coli proteome...
This will take 2-5 minutes for RAST annotation and model construction.

✓ Genome annotated with RAST successfully

✓ Model built: E_coli_K12.draft
  Reactions: 1829
  Metabolites: 1502
  Genes: 1333
  Exchange reactions: 188
  Template: GramNegative
  Compartments: ['c0', 'e0']
  Has biomass reaction: True

Note: This is a DRAFT model (suffix: .draft)
      It likely cannot predict growth without gapfilling.


## Step 3: Gapfill Model

Gapfill the draft model to enable growth in the glucose minimal medium.

Gapfilling adds missing reactions from the template database to enable the model to produce biomass.

This process has two stages:
1. **ATP Correction** - Ensures ATP production pathways work
2. **Genome-Scale Gapfilling** - Adds reactions for target media and growth

In [5]:
print("Gapfilling model...")
print("This may take 1-5 minutes depending on model complexity.\n")

gapfill_response = gapfill_model(
    model_id=model_id,
    media_id=media_id,
    db_index=db_index,
    target_growth_rate=0.01,  # Minimum viable growth
    gapfill_mode="full"       # Full gapfilling (ATP + genome-scale)
)

gapfilled_model_id = gapfill_response["model_id"]
print(f"\n✓ Model gapfilled: {gapfilled_model_id}")
print(f"  Original model: {gapfill_response['original_model_id']}")
print(f"  Reactions added: {gapfill_response['num_reactions_added']}")
print(f"  Growth rate before: {gapfill_response['growth_rate_before']:.3f} hr⁻¹")
print(f"  Growth rate after: {gapfill_response['growth_rate_after']:.3f} hr⁻¹")

# Display statistics
atp_stats = gapfill_response["atp_correction"]
gs_stats = gapfill_response["genomescale_gapfill"]

print(f"\nGapfilling Statistics:")
print(f"  ATP Correction:")
print(f"    Performed: {atp_stats['performed']}")
print(f"    Media tested: {atp_stats['media_tested']}")
print(f"    Media passed: {atp_stats['media_passed']}")
print(f"    Media failed: {atp_stats['media_failed']}")
print(f"    Reactions added: {atp_stats['reactions_added']}")
print(f"  Genome-Scale Gapfilling:")
print(f"    Performed: {gs_stats['performed']}")
print(f"    New reactions: {gs_stats['new_reactions']}")
print(f"    Reversed reactions: {gs_stats['reversed_reactions']}")

# Display added reactions
if gapfill_response["reactions_added"]:
    print(f"\nAdded Reactions (showing first 5):")
    for rxn in gapfill_response["reactions_added"][:5]:
        print(f"  {rxn['id']}: {rxn['name']}")
        print(f"    {rxn['equation']}")
        print(f"    Direction: {rxn['direction']}, Bounds: {rxn['bounds']}")

Gapfilling model...
This may take 1-5 minutes depending on model complexity.



No gapfilling solution found before filtering for Etho activating ATPM_c0
No gapfilling solution found before filtering for mal-L activating ATPM_c0
No gapfilling solution found before filtering for Pyr.SO4 activating ATPM_c0
No gapfilling solution found before filtering for H2.SO4 activating ATPM_c0
No gapfilling solution found before filtering for empty activating ATPM_c0
No gapfilling solution found before filtering for Light activating ATPM_c0
No gapfilling solution found before filtering for ANME activating ATPM_c0
No gapfilling solution found before filtering for Methane activating ATPM_c0



✓ Model gapfilled: E_coli_K12.draft.gf
  Original model: E_coli_K12.draft
  Reactions added: 4
  Growth rate before: 0.000 hr⁻¹
  Growth rate after: 0.562 hr⁻¹

Gapfilling Statistics:
  ATP Correction:
    Performed: True
    Media tested: 54
    Media passed: 27
    Media failed: 1
    Reactions added: 0
  Genome-Scale Gapfilling:
    Performed: True
    New reactions: 5
    Reversed reactions: 0

Added Reactions (showing first 5):
  rxn05459_c0: stearyl-ACP:[acyl-carrier-protein] transferase
    (1) cpd00010[0] + (1) cpd00067[0] + (1) cpd11573[0] <=> (1) cpd00327[0] + (1) cpd11493[0]
    Direction: forward, Bounds: [0, 1000]
  rxn05481_c0: 5-Methylthio-D-ribose transport in/out via proton symport
    (1) cpd00067[1] + (1) cpd01981[1] <=> (1) cpd00067[0] + (1) cpd01981[0]
    Direction: reverse, Bounds: [-1000, 0]
  rxn00599_c0: succinyl-CoA:glycine C-succinyltransferase (decarboxylating)
    (1) cpd00033[0] + (1) cpd00067[0] + (1) cpd00078[0] => (1) cpd00010[0] + (1) cpd00011[0] + (

## Step 4: Run FBA

Execute Flux Balance Analysis to predict metabolic fluxes and growth rate.

FBA optimizes the biomass objective function subject to:
- Stoichiometric constraints (mass balance)
- Thermodynamic constraints (reaction directionality)
- Media constraints (nutrient availability)

In [6]:
print("Running FBA...")fba_response = run_fba(    model_id=gapfilled_model_id,    media_id=media_id,    db_index=db_index,     # Database index for compound/reaction names    objective="bio1",      # Biomass reaction    maximize=True,         # Maximize growth    flux_threshold=1e-6    # Filter small fluxes)# Check if FBA succeeded or returned an errorif not fba_response.get("success", True):    # Error response    print(f"\n❌ FBA Failed")    print(f"  Error: {fba_response.get('error_type', 'Unknown')}")    print(f"  Message: {fba_response.get('message', 'No message')}")    print(f"\nNote: If you see 'MSMedia' object error, restart the kernel and rerun:")    print(f"  Kernel → Restart & Run All")else:    # Success response    print(f"\n✓ FBA Complete")    print(f"  Status: {fba_response['status']}")    print(f"  Objective value (growth rate): {fba_response['objective_value']:.3f} hr⁻¹")    print(f"  Active reactions: {fba_response['active_reactions']}")    print(f"  Total flux: {fba_response['total_flux']:.1f} mmol/gDW/h")    # Display uptake fluxes    print(f"\nUptake Fluxes (top 5):")    for cpd_id, flux_info in list(fba_response["uptake_fluxes"].items())[:5]:        print(f"  {flux_info['name']} ({cpd_id}): {flux_info['flux']:.3f} mmol/gDW/h")    # Display secretion fluxes    print(f"\nSecretion Fluxes (top 5):")    for cpd_id, flux_info in list(fba_response["secretion_fluxes"].items())[:5]:        print(f"  {flux_info['name']} ({cpd_id}): {flux_info['flux']:.3f} mmol/gDW/h")    # Display top internal fluxes    print(f"\nTop Internal Fluxes (showing first 10):")    for flux_info in fba_response["top_fluxes"][:10]:        print(f"  {flux_info['reaction_name']} ({flux_info['reaction_id']}): {flux_info['flux']:.3f}")    # Display summary    summary = fba_response["summary"]    print(f"\nSummary:")    print(f"  Uptake reactions: {summary['uptake_reactions']}")    print(f"  Secretion reactions: {summary['secretion_reactions']}")    print(f"  Internal reactions: {summary['internal_reactions']}")

Running FBA...


TypeError: run_fba() missing 1 required positional argument: 'db_index'

## Interpretation

The FBA results show:

1. **Growth Rate**: The predicted growth rate in hr⁻¹ (reciprocal hours)
   - Typical E. coli values: 0.8-1.0 hr⁻¹ in glucose minimal aerobic

2. **Uptake Fluxes**: Nutrients consumed from the medium
   - Negative flux = uptake/consumption
   - Glucose and oxygen are typically the highest uptakes

3. **Secretion Fluxes**: Metabolic byproducts released
   - Positive flux = secretion/production
   - CO2 is typically the main secretion product in aerobic growth

4. **Internal Fluxes**: Metabolic pathway activity
   - Shows which reactions are active (flux > threshold)
   - High fluxes often indicate key pathways (glycolysis, TCA cycle)

## Workflow Complete!

We have successfully:
1. ✓ Created growth media
2. ✓ Built a draft metabolic model from E. coli proteome
3. ✓ Gapfilled the model for growth
4. ✓ Analyzed flux distribution and growth rate
5. ✓ Exported the final model to JSON

## Model States

Notice the model ID transformations:
- **Draft model**: `E_coli_K12.draft`
- **Gapfilled model**: `E_coli_K12.draft.gf`

The original draft model is preserved in storage, allowing you to compare before/after gapfilling.

## Model Output

The final gapfilled model has been saved to:
- **File**: `examples/output/E_coli_K12.draft.gf.json`
- **Format**: JSON (COBRApy-compatible)
- **Can be loaded with**: `cobra.io.load_json_model()`

## Next Steps

Try modifying this workflow:
- Use different media (anaerobic, pyruvate)
- Try without RAST annotation (`annotate_with_rast=False`)
- Analyze different objective functions
- Compare multiple gapfilling iterations

See other notebooks:
- `02_database_lookups.ipynb` - Explore compound/reaction databases
- `03_session_management.ipynb` - Manage models and media
- `04_error_handling.ipynb` - Handle common errors

## Step 5: Export Model to JSON

Save the final gapfilled model as a JSON file for future use or sharing.

In [ ]:
# Export the final gapfilled model to JSON
from cobra.io import save_json_model
from gem_flux_mcp.storage.models import retrieve_model
from pathlib import Path

# Retrieve the gapfilled model from storage
final_model = retrieve_model(gapfilled_model_id)

# Create output directory
output_dir = Path("examples/output")
output_dir.mkdir(exist_ok=True)

# Save to JSON
output_file = output_dir / f"{gapfilled_model_id}.json"
save_json_model(final_model, str(output_file))

print(f"✓ Model exported to JSON: {output_file}")
print(f"  Model ID: {final_model.id}")
print(f"  Reactions: {len(final_model.reactions)}")
print(f"  Metabolites: {len(final_model.metabolites)}")
print(f"  Genes: {len(final_model.genes)}")
print()
print("You can load this model later with:")
print(f"  from cobra.io import load_json_model")
print(f"  model = load_json_model('{output_file}')")